In [46]:
# EMC2 WP 2 Processing 0.1
# GeoPackage : Department and subset (commune) preparation
# BD_TOPO (Building, roads, administrative limits and activity areas),
# Filosofi (population)
# Author : Perez Joan

# Download : last version of BD_TOPO for a given French department (BD TOPO® décembre 2023 Tous Thèmes par département format GeoPackage projection légale)
# Webpage : https://geoservices.ign.fr/bdtopo#telechargementgpkgreg
# Chosen department : Alpes maritimes (06)
# https://data.geopf.fr/telechargement/download/BDTOPO/BDTOPO_3-3_TOUSTHEMES_GPKG_LAMB93_D006_2023-12-15/BDTOPO_3-3_TOUSTHEMES_GPKG_LAMB93_D006_2023-12-15.7z

# Download : last version of Filosofi (2019)
# https://www.insee.fr/fr/statistiques/7655475?sommaire=7655515

In [47]:
# 0.1 Packages, local filepaths & parameters
import os
import pandas
import geopandas
import time
import contextily
import matplotlib.pyplot as plt

# Set filepaths to downloaded data
BD_TOPO = "C:\\Users\\jperez\\Documents\\Current 1\\emc2\\Raw data\\BDTOPO 3-3\\1_DONNEES_LIVRAISON_2023-12-00191\\BDT_3-3_GPKG_LAMB93_D006-ED2023-12-15\\BDT_3-3_GPKG_LAMB93_D006-ED2023-12-15.gpkg"
Filosofi = "C:\\Users\\jperez\\Documents\\Current 1\\emc2\\Raw data\\Filosofi 2019\\carreaux_200m_met.shp"

# Specify the file path to record the main gpkg file & the subset
main_gpkg = "C:\\Users\\jperez\\Documents\\Current 1\\emc2\\Output\\WP2_DPC_0.1.gpkg"
subset_gpkg = "C:\\Users\\jperez\\Documents\\Current 1\\emc2\\Output\\WP2_DPC_0.1_Subset.gpkg"

# Name of the commune for the subset (example : Drap)
subset = "Drap"

In [48]:
## 1. load raw data

# Read the administrative boundaries
start_time = time.time()
administrative = geopandas.read_file(BD_TOPO, layer="commune", engine='pyogrio', use_arrow=True)
# Read roads
road = geopandas.read_file(BD_TOPO, layer="troncon_de_route", engine='pyogrio', use_arrow=True)
# Read buildings
building = geopandas.read_file(BD_TOPO, layer="batiment", engine='pyogrio', use_arrow=True)
# Read activity areas
activity_area = geopandas.read_file(BD_TOPO, layer="zone_d_activite_ou_d_interet", engine='pyogrio', use_arrow=True)
# Read population
population = geopandas.read_file(Filosofi, engine='pyogrio', use_arrow=True)
end_time = time.time()
processing_time = end_time - start_time
print("Import processing time:", processing_time, "seconds")

Import processing time: 54.316184997558594 seconds


In [49]:
## 2. Subset & clean data

# Keep population squares for given department only
dpt_population = geopandas.sjoin(population, administrative, predicate='within')
dpt_population = dpt_population.iloc[:, list(range(35))]

# Remove variables with wrong format in BD_TOPO datasets (compatibility issues with geopandas
administrative_clean = administrative.iloc[:, list(range(8)) + list(range(12, 19)) + list(range(20, 26))]
building_clean = building.iloc[:, list(range(6)) + list(range(10, 28))]
road_clean = road.iloc[:, list(range(8)) + list(range(12, 44)) + list(range(45, 85))]
activity_area_clean = activity_area.iloc[:, list(range(9)) + list(range(13, 20))]

In [50]:
## 3. Create a subset for a given commune
administrative_clean_subset = administrative_clean[administrative_clean["nom_officiel"] == subset]
dpt_population_subset = geopandas.sjoin(dpt_population, administrative_clean_subset[['geometry']], predicate='intersects')
building_clean_subset = geopandas.sjoin(building_clean, administrative_clean_subset[['geometry']], predicate='intersects')
road_clean_subset = geopandas.sjoin(road_clean, administrative_clean_subset[['geometry']], predicate='intersects')
activity_area_clean_subset = geopandas.sjoin(activity_area_clean, administrative_clean_subset[['geometry']], predicate='intersects')

In [ ]:
## 4. Create GPKG for downloaded department & subset

# Save main gpkg
# Save the GeoDataFrames to a gpkg file
dpt_population.to_file(main_gpkg, layer = "population", driver="GPKG")
building_clean.to_file(main_gpkg, layer = "building", driver="GPKG")
road_clean.to_file(main_gpkg, layer = "road", driver="GPKG")
administrative_clean.to_file(main_gpkg, layer = "administrative", driver="GPKG")
activity_area_clean.to_file(main_gpkg, layer = "activity_areas", driver="GPKG")

# Save subset
# Save the GeoDataFrames to a GeoPackage file
dpt_population_subset.to_file(subset_gpkg, layer = "population", driver="GPKG")
building_clean_subset.to_file(subset_gpkg, layer = "building", driver="GPKG")
road_clean_subset.to_file(subset_gpkg, layer = "road", driver="GPKG")
administrative_clean_subset.to_file(subset_gpkg, layer = "administrative", driver="GPKG")
activity_area_clean_subset.to_file(subset_gpkg, layer = "activity_areas", driver="GPKG")

In [ ]:
# Appendixes A1 : check : population filter (Filosofi) on given department
ax = dpt_population.plot(figsize=(9, 9))
# Add basemap
contextily.add_basemap(ax, crs=dpt_population.crs, source=contextily.providers.CartoDB.Voyager)
# Manually create legend handles and labels
legend_handles = [plt.Rectangle((0, 0), 1, 1, color='lightblue')]
legend_labels = ["Population data available"]
# Add custom legend
ax.legend(legend_handles, legend_labels)
plt.show()

In [ ]:
# Appendixes A2 : check column names & indexes before/after removal of columns
before = road # layer name
for index, column_name in enumerate(before.columns):
    print(f"Column {index}: {column_name}")
after = road_clean # layer name
for index, column_name in enumerate(after.columns):
    print(f"Column {index}: {column_name}")

In [ ]:
# Appendixes A3 : Map of subset
# Create custom legend handles
legend_handles = [
    plt.Rectangle((0, 0), 1, 1, color='black', lw=2, label='Buildings'),  # For buildings
    plt.Line2D([0], [0], color='lightblue', lw=2, label='Roads'), # For roads
    plt.Rectangle((0, 0), 1, 1, color='red', ec='none', label='Activity Areas'),  # For activity areas
    plt.Rectangle((0, 0), 1, 1, fc='orange', ec='none', label='Population data', alpha=0.3)  # For population
]
# Create a plot with building_clean
ax = building_clean_subset.plot(color='black')
# Plot road_clean on the same map
road_clean_subset.plot(ax=ax, linewidth=0.5)
# Plot activity areas on the same map
activity_area_clean_subset.plot(ax=ax, color='red')
# Plot population on the same map
ax = dpt_population_subset.plot(ax=ax, alpha=0.3, color='orange')
# Add title & legend
plt.title(subset)
ax.legend(handles=legend_handles)
# Add basemap
contextily.add_basemap(ax, crs=dpt_population_subset.crs, source=contextily.providers.CartoDB.Voyager)